# Cálculo de Promedio Diario y Calidad del aire

Ahora que ya tenemos nuestros datasets sin outliders, ya podemos calcular nuestro promedio diario y el índice de calidad del aire para cada partícula.

## Carga general de datos

Para obtener nuestros cálculos, utilizaremos las siguientes librerias y funciones:

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Determina el índice de calidad del aire según la metodología y los valores de la tabla descrita en el apartado Metodología
def AQI(df):
    CP = df['PromDiario']
    i = 0

    if 0 <= CP <= 40:
        k = (50-0)/(40-0)
        i = k * (CP - 0) + 0
    if 41 <= CP <= 75:
        k = (100-51)/(75-41)
        i = k * (CP - 41) + 51
    if 76 <= CP <= 214:
        k = (150-101)/(214-76)
        i = k * (CP - 76) + 101
    if 215 <= CP <= 354:
        k = (200-151)/(354-215)
        i = k * (CP - 215) + 151
    if 355 <= CP <= 424:
        k = (300-201)/(242-355)
        i = k * (CP - 355) + 201
    if 425 <= CP <= 504:
        k = (400-301)/(504-425)
        i = k * (CP - 425) + 301
    if 505 <= CP <= 604:
        k = (500-401)/(604-505)
        i = k * (CP - 505) + 401
    
    return round(i, 3)

In [3]:
# Determina la categoría de calidad del aire descrita por la norma mexicana
def AQI_categoria(df):
    indice = df['Indice']
    if indice <= 50:
      clase='Buena'
    elif indice <= 100:
      clase='Regular'
    elif indice <= 150:
      clase= 'Mala'
    elif indice <= 200:
      clase = 'Muy mala'
    else: 
      clase = 'Extremadamente mala'
    
    return clase

In [4]:
def dia_estacion_valido(dataframe):
  #Contar las mediciones horarias por día y estación
  filtro= dataframe.groupby(['FECHA', 'station'],as_index=False).agg(N_h=('HORA','count'))
  #Filtrar aquellas que no cumplen normativa
  filtro= filtro.query('N_h < 17')
  #Conservar las columnas de fecha y estación
  filtro=filtro.loc[:,['FECHA','station']]
  #Eliminar del dataframe original las mediciones que no cumplen normativa
  filtro=pd.merge(dataframe,filtro, how='outer', indicator=True)
  filtro=filtro[filtro['_merge'] == 'left_only']
  return filtro

Vamos a realizar lo siguiente:

- Cargar los documentos que validaremos, para hacer los respectivos cálculos

In [5]:
#Cargando archivos
df_PM10 = pd.read_csv('../Datasets/PM10/PM10_depurado_melt_no_outlier.csv')
df_PM25 = pd.read_csv('../Datasets/PM2.5/PM25_depurado_melt_no_outlier.csv')

# Normalizando fechas
df_PM10['FECHA'] = pd.to_datetime(df_PM10['FECHA'], unit='ns')
df_PM25['FECHA'] = pd.to_datetime(df_PM25['FECHA'], unit='ns')

## Cálculos $PM_{10}$

Ahora se busca eliminar, según el día y la estación, aquellas filas que tengan mediciones horarias menores a las requeridas por la normativa (18 mediciones). Por ejemplo, si la estación ACO en el 01 de enero de 2019 registró solamente 15 mediciones horarias, todas las mediciones de ese día para la estación ACO no será tomadas en cuenta para los posteriores análisis. 

In [6]:
df_filtrado= dia_estacion_valido(df_PM10)
df_filtrado = df_filtrado.drop(columns=['_merge'])
df_filtrado=df_filtrado.reset_index()
df_filtrado = df_filtrado.drop(columns=['index'])
df_filtrado.head()

,Unnamed: 0,FECHA,Year,Month,HORA,station,measurement,Zona
0,1,2019-03-01,2019,3,2,ACO,90.0,NE
1,2,2019-03-01,2019,3,3,ACO,81.0,NE
2,3,2019-03-01,2019,3,4,ACO,65.0,NE
3,4,2019-03-01,2019,3,5,ACO,53.0,NE
4,5,2019-03-01,2019,3,6,ACO,52.0,NE


### Calculando el promedio diario
Para el cálculo del promedio diario de $PM_{10}$ se utilizó la metodología descrita en la `NOM-025-SSA1-2014`, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores mínimos y máximos

In [7]:
df_PM1024h = df_filtrado.groupby(['FECHA', 'Year', 'Month', 'Zona'], as_index=False).agg(
            PromDiario=('measurement', lambda x: round(x.mean(),0)), 
            Max = ('measurement', 'max'),
            Min = ('measurement', 'min'))
df_PM1024h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min
0,2019-03-01,2019,3,CE,56.0,101.0,23.0
1,2019-03-01,2019,3,NE,81.0,161.0,37.0
2,2019-03-01,2019,3,NO,63.0,112.0,21.0
3,2019-03-01,2019,3,SE,56.0,111.0,8.0
4,2019-03-01,2019,3,SO,38.0,85.0,3.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C - BP_{Lo}))+I_{Lo}$$
$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$
 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>

In [8]:
df_PM1024h['Indice'] = df_PM1024h.apply(AQI, axis=1)
df_PM1024h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min,Indice
0,2019-03-01,2019,3,CE,56.0,101.0,23.0,72.618
1,2019-03-01,2019,3,NE,81.0,161.0,37.0,102.775
2,2019-03-01,2019,3,NO,63.0,112.0,21.0,82.706
3,2019-03-01,2019,3,SE,56.0,111.0,8.0,72.618
4,2019-03-01,2019,3,SO,38.0,85.0,3.0,47.500


Se aplica la función para determinar la categoría a cada promedio diario calculado.

In [9]:
df_PM1024h['Clase'] = df_PM1024h.apply(AQI_categoria, axis=1)
df_PM1024h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min,Indice,Clase
0,2019-03-01,2019,3,CE,56.0,101.0,23.0,72.618,Regular
1,2019-03-01,2019,3,NE,81.0,161.0,37.0,102.775,Mala
2,2019-03-01,2019,3,NO,63.0,112.0,21.0,82.706,Regular
3,2019-03-01,2019,3,SE,56.0,111.0,8.0,72.618,Regular
4,2019-03-01,2019,3,SO,38.0,85.0,3.0,47.500,Buena


Se guardan los resultados en un csv

In [10]:
df_PM1024h.to_csv('../Datasets/PM10/PDZona_PM10.csv', index=False)

## Cálculos $PM_{2.5}$

Ahora se busca eliminar, para está partícula, aquellas filas que tengan mediciones horarias menores a las requeridas por la normativa (18 mediciones).

In [11]:
df_filtrado= dia_estacion_valido(df_PM25)
df_filtrado = df_filtrado.drop(columns=['_merge'])
df_filtrado=df_filtrado.reset_index()
df_filtrado = df_filtrado.drop(columns=['index'])
df_filtrado.head()

,Unnamed: 0,FECHA,Year,Month,HORA,station,measurement,Zona
0,0,2019-03-01,2019,3,1,AJM,8.0,SO
1,1,2019-03-01,2019,3,2,AJM,19.0,SO
2,2,2019-03-01,2019,3,3,AJM,29.0,SO
3,3,2019-03-01,2019,3,4,AJM,28.0,SO
4,4,2019-03-01,2019,3,5,AJM,26.0,SO


### Calculando el promedio diario
Para el cálculo del promedio diario de $PM_{2.5}$ se utilizó la metodología descrita en la `NOM-025-SSA1-2014`, siguiendo la siguiente ecuación:

$$\bar{x}= \frac{1}{n} \displaystyle\sum_{i=1}^n x_i$$
$\bar{x}:$ promedio de 24 horas, 

$n:$ número de concentraciones horarias válidas

$x_i:$ concentraciones horarias válidas

- Camo datos extra obteniendo los valores mínimos y máximos

In [12]:
df_PM2524h = df_filtrado.groupby(['FECHA', 'Year', 'Month', 'Zona'], as_index=False).agg(PromDiario=('measurement', lambda x: round(x.mean(),1)), #En este caso, se redondea a un decimal
                                                                    Max = ('measurement', 'max'),
                                                                    Min = ('measurement', 'min'))
df_PM2524h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min
0,2019-03-01,2019,3,CE,27.0,55.0,8.0
1,2019-03-01,2019,3,NE,32.2,62.0,1.0
2,2019-03-01,2019,3,NO,35.1,61.0,12.0
3,2019-03-01,2019,3,SE,24.4,47.0,6.0
4,2019-03-01,2019,3,SO,20.3,44.0,1.0


### Calculando Índice de calidad del aire

Donde:
$$\text{Índice}= (k \times (C - BP_{Lo}))+I_{Lo}$$
$$k= \frac{I_{Hi}-I_{Lo}}{BP_{Hi}-BP_{Lo}}$$
 
$\text{Índice}:$ Valor del índice para el contaminante deseado.<br>
$C:$ valor redondeado para la concentración del contaminante.<br>
$k:$ constante de proporcionalidad estimada.<br>
$BP_{Hi}:$ valor del punto de corte que es mayor o igual a la concentración a evaluar.<br>
$BP_{Lo}:$ valor del punto de corte que es menor o igual a la concentración a evaluar.<br>
$I_{Hi}:$ valor del índice que corresponde al valor de $BP_{Hi}$<br>
$I_{Lo}:$ valor del índice que corresponde al valor de $BP_{Lo}$<br>

In [13]:
df_PM2524h['Indice'] = df_PM2524h.apply(AQI, axis=1)
df_PM2524h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min,Indice
0,2019-03-01,2019,3,CE,27.0,55.0,8.0,33.750
1,2019-03-01,2019,3,NE,32.2,62.0,1.0,40.250
2,2019-03-01,2019,3,NO,35.1,61.0,12.0,43.875
3,2019-03-01,2019,3,SE,24.4,47.0,6.0,30.500
4,2019-03-01,2019,3,SO,20.3,44.0,1.0,25.375


Se aplica la función para determinar la categoría a cada promedio diario calculado.

In [14]:
df_PM2524h['Clase'] = df_PM2524h.apply(AQI_categoria, axis=1)
df_PM2524h.head()

,FECHA,Year,Month,Zona,PromDiario,Max,Min,Indice,Clase
0,2019-03-01,2019,3,CE,27.0,55.0,8.0,33.750,Buena
1,2019-03-01,2019,3,NE,32.2,62.0,1.0,40.250,Buena
2,2019-03-01,2019,3,NO,35.1,61.0,12.0,43.875,Buena
3,2019-03-01,2019,3,SE,24.4,47.0,6.0,30.500,Buena
4,2019-03-01,2019,3,SO,20.3,44.0,1.0,25.375,Buena


Se guardan los resultados en un csv

In [15]:
df_PM2524h.to_csv('../Datasets/PM2.5/PDZona_PM25.csv', index=False)